In [1]:
from app.dataprocessing.data_handler import DataHandler

data_handler = DataHandler()
data_handler.set_max_chunk_size(1)

file_path = 'app/externalresources/datasets/GS17_ROV01_LokiCastle_08072017_LLD_depthcorrected.nc'

data_handler.set_local_netcdf_reader(file_path)


print(f"Data structure in use: {data_handler.data_structure}")
data_handler.ds


Finished 'Loading dataset' in 0.1121 seconds
Finished 'Creating data structure' in 2.3546 seconds
Data structure in use: QuadTree with 64 chunks at lowest level of max chunk size 1MB


<xarray.Dataset>
Dimensions:  (lon: 12545, lat: 3205)
Coordinates:
  * lon      (lon) float64 8.146 8.146 8.146 8.147 ... 8.169 8.169 8.169 8.169
  * lat      (lat) float64 73.56 73.56 73.56 73.56 ... 73.57 73.57 73.57 73.57
Data variables:
    z        (lat, lon) float32 ...
Attributes:
    Conventions:  COARDS, CF-1.5
    title:        
    history:      nearneighbor GS17_ROV01_LokiCastle_08072017_LLD_depthcorrec...
    description:  
    GMT_version:  5.4.1 (r18208) [64-bit]

In [2]:
from app.dataprocessing.benchmark import plt_img, save_simple_img
from app.datastructures.datastructure_interface import get_ipyleaflet_bounds

initial_ds, bnds, node = data_handler.get_initial_ds()

data_variable = "z"
arr = initial_ds[data_variable].isel(
    {x: 0 for x in initial_ds.dims if x not in ["lat", "lon", "latitude", "longitude"]}
)
vmin_glob = arr.min().values
vmax_glob = arr.max().values

inital_render = plt_img(arr, vmin_glob, vmax_glob)
inital_render = save_simple_img(initial_ds, data_variable, vmin_glob, vmax_glob)

ipy_bnds = get_ipyleaflet_bounds(node.ds)
print(ipy_bnds)


((73.563701, 8.1464949), (73.5694677, 8.169067699744899))


In [5]:
from ipyleaflet import basemaps, Map, ImageOverlay, projections
import ipywidgets
import numpy as np
from app.dataprocessing.benchmark import Stopwatch
from app.datastructures.datastructure_interface import spatial_resolution


def print_bounds(bounds):
    sw, ne = bounds
    s, w = sw
    n, e = ne
    return f"SW: ({s:.2f}, {w:.2f}) NE: ({n:.2f}, {e:.2f})"


# bm = basemaps.Esri.WorldTopoMap.from_qms
# bm.projections = ''
button = ipywidgets.Button(description="Render")
button2 = ipywidgets.Button(description="Clear Layers")
button3 = ipywidgets.Button(description="Render, new colors")
# m = Map(crs=projections.EPSG4326, center=[0, 0], zoom=0)
m = Map(crs=projections.EPSG4326)
m.clear_layers()  # remove default map

image = ImageOverlay(url=inital_render, bounds=ipy_bnds, opacity=1)
m.add_layer(image)

m.fit_bounds(ipy_bnds)
m.max_zoom = 60


output = ipywidgets.Output()
with output:
    print(f"Initial render at {data_handler.get_node_resolution(node):.2f}% resolution")
    print(
        f"Dataset size: {data_handler.get_file_size_MB(file_path):.2f} MB, Max chunk size: {data_handler.max_chunk_size} MB"
    )
    print(f"Bounds: {print_bounds(bnds)}")
    print(f"-----------------------------------------------------")
# display(button, button2, m, output)

display(ipywidgets.HBox((button, button3, button2)), m, output)


def handle_map_interaction(**kwargs):
    if kwargs.get("type") == "mouseup":
        with output:
            print(m.bounds)


def render(reset_color_scale=False):
    with output:
        timer = Stopwatch()

        print(f"Query bounds: {print_bounds(m.bounds)}")

        timer.start("Fetch netCDF chunk")
        sw_ne = m.bounds

        query_bounds = ((sw_ne[0][0], sw_ne[1][0]), (sw_ne[0][1], sw_ne[1][1]))
        # (np.datetime64("2021-09-23"), np.datetime64("2021-09-25")),

        file_name, bounds, node = data_handler.request_data_netcdf(
            query_bounds, return_xr_chunk=True
        )
        fetch_time = timer.stop()

        timer.start("Rendering")
        respons_bounds_sw_ne = (
            (bounds[0][0], bounds[1][0]),
            (bounds[0][1], bounds[1][1]),
        )

        if reset_color_scale:
            vmin = arr.min().values
            vmax = arr.max().values
        else:
            vmin = vmin_glob
            vmax = vmax_glob

        render = save_simple_img(node.ds, data_variable, vmin, vmax)

        # change opacity on old layers
        for l in m.layers:
            if isinstance(l, ImageOverlay):
                l.opacity = 0.8

        image = ImageOverlay(url=render, bounds=respons_bounds_sw_ne, opacity=1)
        m.add_layer(image)
        render_time = timer.stop()

        print(f"Chunk bounds: {print_bounds(respons_bounds_sw_ne)}")
        print(
            f"Chunk resolution: {data_handler.get_node_resolution(node):.2f}%, File size: {data_handler.get_file_size_MB(file_name):.2f} MB"
        )
        print(f"Spatial resolution: {spatial_resolution(node.ds)}")
        print(f"{fetch_time}")
        print(f"{render_time}")
        print(f"-----------------------------------------------------")


def clear_map(event):
    m.clear_layers()


def update_map(event):
    render(reset_color_scale=False)


def reset_vmin_vmax(event):
    render(reset_color_scale=True)


button.on_click(update_map)
button2.on_click(clear_map)
button3.on_click(reset_vmin_vmax)
# m.on_interaction(handle_map_interaction)


Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Output()